Importar bibliotecas

In [0]:
import os
import pyspark.sql.functions as F

Motor ingestão raw

In [0]:
landing_path = "/Volumes/taxi_catalog/landing/yellow_taxi"
bronze_path = "taxi_catalog.bronze.yellow_taxi"

# caminho base dos arquivos
files = [f.path for f in dbutils.fs.ls(landing_path) if f.path.endswith(".parquet")]

df_list = []

# iteração para cada arquivo no file
for file_path in files:
    df_temp = (
        spark.read.parquet(file_path)
        .withColumn("source_file", F.lit(file_path))
        .withColumn("ingestion_ts", F.current_timestamp())
    )

    # cast de tipos que podem gerar conflito
    df_temp = (
        df_temp.withColumn("passenger_count", F.col("passenger_count").cast("double"))
        .withColumn("VendorID", F.col("VendorID").cast("long"))
        .withColumn("RatecodeID", F.col("RatecodeID").cast("double"))
        .withColumn("PULocationID", F.col("PULocationID").cast("long"))
        .withColumn("DOLocationID", F.col("DOLocationID").cast("long"))
    )

    df_list.append(df_temp)

df_bronze = df_list[0]
for df in df_list[1:]:
    df_bronze = df_bronze.unionByName(df, allowMissingColumns=True)

df_bronze.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(bronze_path)